In [1]:
# Import images from Websites
# Article
# https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d

In [2]:
# Import necessary librariers to run the code
import time
import requests
import io
import hashlib
import os

# This is helpful to create images
from PIL import Image, ImageDraw

# This part is used to automate the process of downloading the dataset
#!pip install selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [3]:
# pip install webdriver-manager
# import selenium
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.by import By
# # This is the path I use
# # DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# # Put the path for your ChromeDriver here

# DRIVER_PATH = "D:/Projects/CelebrityImageClassification/chromedriver/chromedriver"
# # s = Service(ChromeDriverManager().install())
# s = Service(DRIVER_PATH)
# wd = webdriver.Chrome(service=s)
# wd.maximize_window()
# wd.get('https://www.google.com')
# wd.find_element(By.NAME, 'q').send_keys('Yasser Khalil\n')

In [4]:
def fetch_image_url_util(url, driver_path):
    images = []
    
    # Open main window with URL A
    with webdriver.Chrome(service=Service(driver_path)) as wd:
        # Oepn URL B in new window
        try:
            wd.get(url)
        except:
            return []
        
        
        # thumbnail_results = wd.find_elements_by_css_selector("img[class ='irc_mi']")
        # thumbnail_results = wd.find_elements(by=By.CSS_SELECTOR, value="img[class ='irc_mi']")
        thumbnail_results = wd.find_elements(by=By.CSS_SELECTOR, value="img.n3VNCb")
        
        for img in thumbnail_results:
            if img.get_attribute('src') and 'http' in img.get_attribute('src'):
                images.append(img.get_attribute('src'))
                
        return images

In [5]:
def scroll_to_end(wd, sleep_between_interactions):
    """
    This function is used to scroll the webpage
    And wait for some time before scrolling using the time.sleep method
    """
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(sleep_between_interactions)

In [6]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd, sleep_between_interactions:int=1, driver_path=None, target_path=None, search_term=None):
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' ')))
    
    # Make the Google Link
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    
    # Load the page
    wd.get(search_url.format(q=query))
    
    image_urls = set()
    image_urls_all = set()
    image_count = 0
    image_count2 = 0
    results_start = 0
    i = 0
    d = {}
    
    while image_count < max_links_to_fetch:
        scroll_to_end(wd, sleep_between_interactions)
        
        # Get the Thumbnails of the images from a webpage
        thumbnail_result = wd.find_elements(by=By.CSS_SELECTOR, value="img.Q4LuWd")
        number_results = len(thumbnail_result)
        
        # print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_result:
            # Try and get the automated click and get the image from the query
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception as e:
                print(e)
                continue

            links = wd.find_elements(by=By.CSS_SELECTOR, value="a[jsname='sTFXNd']")

            for link in links:
                if link.get_attribute('href') and 'http' in link.get_attribute('href'):
                    if link.get_attribute('href') not in d:
                        d[link.get_attribute('href')] = True
                        getactualurl = fetch_image_url_util(link.get_attribute('href'), driver_path)
                    for image_url in getactualurl:
                        if (image_url is not None) and (image_url not in image_urls_all):
                            image_urls.add(image_url)
                            image_urls_all.add(image_url)
                            
            # print(image_urls)
            image_count2 = len(image_urls)
            # print(f"Image Count2: {image_count2}")

            if image_count2 >= max_links_to_fetch/2:
                # print(f"Found: {len(image_urls)} image links, saving!")
                try:
                    for elements in image_urls:
                        persist_image(target_folder, elements)
                except Exception as e:
                    print(e)
                image_urls = set()
                d = {}

                image_count = image_count + image_count2
            if image_count >= max_links_to_fetch:
                break
            
        #image_count = len(image_urls)
        # print(f"Len(image_urls): {len(image_urls)}")
        if image_count >= max_links_to_fetch:
            # print(f"Found: {image_count} image links, done!")
            break
        else:
            # print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            load_more_button = wd.find_elements(by=By.CSS_SELECTOR, value=".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")
        
        # Move the result startpoint further doen
        results_start = image_count
        
    return image_urls

In [7]:
def persist_image(folder_path:str, url:str):
    try:
        image_content = requests.get(url).content
    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")
        
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path, hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, "wb") as f:
            image.save(f, "JPEG", quality=85)
        # print(f"SUCCESS - saved {url} - as {file_path}")
        # print(f"SUCCESS - saved.")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [8]:
def search_and_download(search_term:str, driver_path:str, target_path="../dataset", number_images=5):
    target_folder = os.path.join(os.path.abspath(target_path), '_'.join(search_term.lower().split(' ')))
    
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        
    with webdriver.Chrome(service=Service(driver_path)) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5,driver_path= driver_path,target_path= target_path,search_term=search_term)
    try:    
        for elem in res:
            persist_image(target_folder,elem)
    except Exception as e:
        print("Search: ", e)

In [9]:
query = ["Virat Kohli", "MS Dhoni", "Sachin Tendulkar", "Rohit Sharma", "Ravindra Jadeja"]

for q in query:
    search_and_download(q, os.path.abspath("../chromedriver/chromedriver.exe"), number_images=50)
    print("Images of ", q, " saved.")

Images of  Virat Kohli  saved.
Images of  MS Dhoni  saved.
Images of  Sachin Tendulkar  saved.
ERROR - Could not save https://indiandailypost.com/wp-content/uploads/2021/12/Rohit-Sharma-6.jpg - cannot identify image file <_io.BytesIO object at 0x000001F4E1F80C20>
Images of  Rohit Sharma  saved.
Images of  Ravindra Jadeja  saved.
